# Introduction
Please first read the notebook 'omnidrone_dynamics.ipynb'. This notebook investigates the advantage of defining the nominal configuration differently to deal with the singularity on $q_{2}=0$ in the original definition.

In [11]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3 = sp.symbols('x_b y_b z_b yaw pitch roll q_1 q_2 q_3', real=True)
state_vector = sp.Matrix([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

# Secondary states
x_be, y_be, z_be = sp.symbols('x^{b}_{be} y^{b}_{be} z^{b}_{be}')

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

In [34]:
Rz = sp.Matrix([
    [sp.cos(yaw), -sp.sin(yaw), 0],
    [sp.sin(yaw),  sp.cos(yaw), 0],
    [0,            0,           1]
])

Ry = sp.Matrix([
    [sp.cos(pitch),  0, sp.sin(pitch)],
    [0,              1, 0            ],
    [-sp.sin(pitch), 0, sp.cos(pitch)]
])

Rx = sp.Matrix([
    [1, 0,            0           ],
    [0, sp.cos(roll), -sp.sin(roll)],
    [0, sp.sin(roll),  sp.cos(roll)]
])

# Intrinsic ZYX: Postmultiplication
R_Ib = Rz * Ry * Rx
sp.pprint(R_Ib)

T_euler = sp.Matrix([[0, -sp.sin(yaw), sp.cos(yaw)*sp.cos(pitch)],
                     [0, sp.cos(yaw), sp.sin(yaw)*sp.cos(pitch)],
                     [1, 0, -sp.sin(pitch)]])

T_A = sp.Matrix([[sp.eye(3), sp.zeros(3, 3)],
                 [sp.zeros(3, 3), T_euler]])

⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)                     sin(roll)⋅cos(pitch)                      ↪

↪ pitch)⋅cos(roll)⋅cos(yaw) + sin(roll)⋅sin(yaw)⎤
↪                                               ⎥
↪ pitch)⋅sin(yaw)⋅cos(roll) - sin(roll)⋅cos(yaw)⎥
↪                                               ⎥
↪            cos(pitch)⋅cos(roll)               ⎦


In [57]:
# Body frame b: FRD at vehicle CoM
T_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x_b],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y_b],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z_b],
                  [0, 0, 0, 1]])

# Frame 0: Z axis of rotation pointing forward, X axis pointing up, Y axis pointing right (90 deg rotation around Yb)
# Rotation is to 0 from b
T_b0 = sp.Matrix([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]])

# Frame 1: Rotate from 0 around z0 by q1, then rotate around x0 by -pi/2
rot_z0 = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0],
                    [sp.sin(q1), sp.cos(q1), 0],
                    [0, 0, 1]])
rot_x0 = sp.Matrix([[1, 0, 0],
                    [0, sp.cos(-sp.pi/2), -sp.sin(-sp.pi/2)],
                    [0, sp.sin(-sp.pi/2), sp.cos(-sp.pi/2)]])
rot_z0x0 = rot_z0 * rot_x0 # postmultiply because y0 is relative axis
T_01 = sp.diag(rot_z0x0, 1)
T_01[0,3] = L1*sp.cos(q1) # Location of origin of 1 in frame 0
T_01[1,3] = L1*sp.sin(q1) # Location of origin 1 in frame 0
T_01 = sp.trigsimp(sp.expand(T_01))
print("Transformation matrix T_01")
sp.pprint(T_01)

# Frame 2: Rotate about z by q2, then about z by -pi/2, and finally about x by pi/2
rot_z1 = sp.Matrix([[sp.cos(q2), -sp.sin(q2), 0],
                    [sp.sin(q2), sp.cos(q2), 0],
                    [0, 0, 1]])
rot_y1_prime = sp.Matrix([[sp.cos(-sp.pi/2), 0, sp.sin(-sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(-sp.pi/2), 0, sp.cos(-sp.pi/2)]])
rot_z1_primeprime = sp.Matrix([[-sp.cos(sp.pi/2), -sp.sin(-sp.pi/2), 0],
                    [sp.sin(-sp.pi/2), sp.cos(-sp.pi/2), 0],
                    [0, 0, 1]])
rot_z1z1x1 = rot_z1 * rot_y1_prime * rot_z1_primeprime
T_12 = sp.diag(rot_z1z1x1, 1)
T_12[0, 3] = L2*sp.sin(q2)
T_12[1, 3] = -L2*sp.cos(q2) # Location of 2 in frame 1
print("Transformation matrix T_12")
sp.pprint(T_12) 

# Frame 3
rot_z2 = sp.Matrix([[sp.cos(q3), -sp.sin(q3), 0],
                    [sp.sin(q3), sp.cos(q3), 0],
                    [0, 0, 1]])

rot_z2 = rot_z2 
T_23 = sp.diag(rot_z2, 1)
T_23[0, 3] = L3*sp.cos(q3)
T_23[1, 3] = L3*sp.sin(q3) # Location of 3 in frame 2
print("Transformation matrix T_23")
sp.pprint(T_23)

# Frame e: Translation along Y3 by L3, rotation about X3 by -pi/2
# This yields the final frame e with positive Z pointing out, X pointing up and Y completing
T_3e = sp.Matrix([[ 0,  0, 1, 0],
                  [-1,  0, 0, 0],
                  [ 0, -1, 0, 0],
                  [ 0,  0, 0, 1]])

T_be = sp.trigsimp(sp.expand(T_b0*T_01*T_12*T_23*T_3e))
    
# Set parameters
T_be = T_be.subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

Transformation matrix T_01
⎡cos(q₁)  0   -sin(q₁)  L₁⋅cos(q₁)⎤
⎢                                 ⎥
⎢sin(q₁)  0   cos(q₁)   L₁⋅sin(q₁)⎥
⎢                                 ⎥
⎢   0     -1     0          0     ⎥
⎢                                 ⎥
⎣   0     0      0          1     ⎦
Transformation matrix T_12
⎡sin(q₂)   0  -cos(q₂)  L₂⋅sin(q₂) ⎤
⎢                                  ⎥
⎢-cos(q₂)  0  -sin(q₂)  -L₂⋅cos(q₂)⎥
⎢                                  ⎥
⎢   0      1     0           0     ⎥
⎢                                  ⎥
⎣   0      0     0           1     ⎦
Transformation matrix T_23
⎡cos(q₃)  -sin(q₃)  0  L₃⋅cos(q₃)⎤
⎢                                ⎥
⎢sin(q₃)  cos(q₃)   0  L₃⋅sin(q₃)⎥
⎢                                ⎥
⎢   0        0      1      0     ⎥
⎢                                ⎥
⎣   0        0      0      1     ⎦


In [55]:
# Denavit-Hartenberg approach
# Parameters
DH_theta = [q1, q2-sp.pi/2, q3]
DH_alpha = [-sp.pi/2, sp.pi/2, 0]
DH_r = [L1, L2, L3]
DH_d = [0, 0, 0]

# Frame 0: X pointing right, Z pointing forward, Y pointing down (i.e. rotation around Zb by pi/2, then rotation around Xb' by pi/2)
# I.e. intrinisic rotation so multiply Rz * Rx'
# Frame 1: Attached at Q2, Z pointing down, X pointing right (rotation around X0 of -pi/2, then translation along x0' by L1)

H_01 = sp.Matrix([[sp.cos(DH_theta[0]), -sp.sin(DH_theta[0])*sp.cos(DH_alpha[0]), sp.sin(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.cos(DH_theta[0])],
                  [sp.sin(DH_theta[0]), sp.cos(DH_theta[0])*sp.cos(DH_alpha[0]), -sp.cos(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.sin(DH_theta[0])],
                  [0, sp.sin(DH_alpha[0]), sp.cos(DH_alpha[0]), DH_d[0]],
                  [0, 0, 0, 1]])

# Frame 2: Attached at Q3, Z pointing forward, X pointing right (rotation around X1 of pi/2, then translation along x1' by L2)
H_12 = sp.Matrix([[sp.cos(DH_theta[1]), -sp.sin(DH_theta[1])*sp.cos(DH_alpha[1]), sp.sin(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.cos(DH_theta[1])],
                  [sp.sin(DH_theta[1]), sp.cos(DH_theta[1])*sp.cos(DH_alpha[1]), -sp.cos(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.sin(DH_theta[1])],
                  [0, sp.sin(DH_alpha[1]), sp.cos(DH_alpha[1]), DH_d[1]],
                  [0, 0, 0, 1]])

# Frame 3: Attached at EE, Z pointing forward, X pointing right (translation along x2 by L3)
H_23 = sp.Matrix([[sp.cos(DH_theta[2]), -sp.sin(DH_theta[2])*sp.cos(DH_alpha[2]), sp.sin(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.cos(DH_theta[2])],
                  [sp.sin(DH_theta[2]), sp.cos(DH_theta[2])*sp.cos(DH_alpha[2]), -sp.cos(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.sin(DH_theta[2])],
                  [0, sp.sin(DH_alpha[2]), sp.cos(DH_alpha[2]), DH_d[2]],
                  [0, 0, 0, 1]])

# Transformation between b and 0: rotation about y by pi/2 then rotation about z by pi/2
R_b0 = sp.Matrix([[sp.cos(sp.pi/2), 0, sp.sin(sp.pi/2)], # Ry by pi/2
                 [0, 1, 0],
                 [-sp.sin(sp.pi/2), 0, sp.cos(sp.pi/2)]]) *  \
        sp.Matrix([[sp.cos(sp.pi/2), -sp.sin(sp.pi/2), 0],  # Rz by pi/2
                  [sp.sin(sp.pi/2), sp.cos(sp.pi/2), 0],
                  [0, 0, 1]])
assert R_b0 == sp.Matrix([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
H_b0 = sp.diag(R_b0, 1)

# Transformation between 3 and e - To align e with frame e from direct derivation
R_3e = sp.Matrix([[sp.cos(sp.pi/2), 0, sp.sin(sp.pi/2)], # Ry by pi/2
                 [0, 1, 0],
                 [-sp.sin(sp.pi/2), 0, sp.cos(sp.pi/2)]]) * \
        sp.Matrix([[sp.cos(-sp.pi/2), -sp.sin(-sp.pi/2), 0],  # Rz by -pi/2
                  [sp.sin(-sp.pi/2), sp.cos(-sp.pi/2), 0],
                  [0, 0, 1]])
assert R_3e == sp.Matrix([[0, 0, 1], [-1, 0, 0], [0, -1, 0]])
H_3e = sp.diag(R_3e, 1)
H_be = (H_b0 * H_01 * H_12 * H_23 * H_3e).subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [64]:
# Testing if the DH formulation yielded the same result as the direct derivation
if not ( H_be == T_be):
    # Test with random inputs
    q1_val = np.random.rand()
    q2_val = np.random.rand()
    q3_val = np.random.rand()
    x_b_val = np.random.rand()
    y_b_val = np.random.rand()
    z_b_val = np.random.rand()
    yaw_val = np.random.rand()
    pitch_val = np.random.rand()
    roll_val = np.random.rand()
    H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val), (x_b, x_b_val), (y_b, y_b_val), (z_b, z_b_val), (yaw, yaw_val), (pitch, pitch_val), (roll, roll_val)])
    T_be_subs = T_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val), (x_b, x_b_val), (y_b, y_b_val), (z_b, z_b_val), (yaw, yaw_val), (pitch, pitch_val), (roll, roll_val)])
    sp.pprint(H_be_subs)
    sp.pprint(T_be_subs)
    
else:
    print("Kinematics verified")

# Unit test 1
q1_val = sp.pi/2
q2_val = 0
q3_val = 0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
T_be_subs = T_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
if (H_be_subs[0,3] == T_be_subs[0,3] and H_be_subs[1,3] == T_be_subs[1,3] and H_be_subs[2,3] == T_be_subs[2,3]):
    print("Unit test 1 passed")
else:
    print("Unit test 1 failed")

# Unit test 2
q1_val = 0
q2_val = 0
q3_val = -sp.pi/2
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
T_be_subs = T_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
if (H_be_subs[0,3] == T_be_subs[0,3] and H_be_subs[1,3] == T_be_subs[1,3] and H_be_subs[2,3] == T_be_subs[2,3]):
    print("Unit test 2 passed")
else:
    print("Unit test 2 failed")

# Unit test 3
q1_val = 0
q2_val = sp.pi/2
q3_val = 0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
T_be_subs = T_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
sp.pprint(H_be_subs)
if (H_be_subs[0,3] == T_be_subs[0,3] and H_be_subs[1,3] == T_be_subs[1,3] and H_be_subs[2,3] == T_be_subs[2,3]):
    print("Unit test 3 passed")
else:
    print("Unit test 3 failed")

⎡0.285210215869334   -0.0839142372379837   0.954784548237211   0.5705592785921 ↪
⎢                                                                              ↪
⎢0.438202852240464    0.897368981883055   -0.0520304780152094  0.1083574778271 ↪
⎢                                                                              ↪
⎢-0.852427940093525   0.433228936179171    0.292710259138578   0.1390798950020 ↪
⎢                                                                              ↪
⎣        0                    0                    0                   1       ↪

↪ 2 ⎤
↪   ⎥
↪ 32⎥
↪   ⎥
↪ 98⎥
↪   ⎥
↪   ⎦
⎡0.285210215869334   -0.0839142372379837   0.954784548237211   0.5705592785921 ↪
⎢                                                                              ↪
⎢0.438202852240464    0.897368981883055   -0.0520304780152094  0.1083574778271 ↪
⎢                                                                              ↪
⎢-0.852427940093525   0.433228936179171    0.292710259138578   0.1

In [66]:
# Forward kinematics function of the manipulator (in the body frame)
FK_manipulator = sp.Matrix([T_be[0,3],
                T_be[1,3],
                T_be[2,3],
                sp.atan2(T_be[1,0], T_be[0,0]), # yaw
                -sp.asin(T_be[2,0]), # pitch
                sp.atan2(T_be[2,1], T_be[2,2])]) # roll

FK_manipulator = sp.simplify(sp.trigsimp(FK_manipulator))
print("Forward kinematics of the manipulator (in the body frame)")
sp.pprint(FK_manipulator)
FK_manipulator_subs = FK_manipulator.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)])
print("Forward kinematics of the manipulator (in the body frame) with q1=pi/2, q2=0, q3=0")
sp.pprint(FK_manipulator_subs)

Forward kinematics of the manipulator (in the body frame)
⎡                               (0.273⋅cos(q₃) + 0.311)⋅cos(q₂)                ↪
⎢                                                                              ↪
⎢-0.273⋅sin(q₁)⋅sin(q₃) + 0.273⋅sin(q₂)⋅cos(q₁)⋅cos(q₃) + 0.311⋅sin(q₂)⋅cos(q₁ ↪
⎢                                                                              ↪
⎢0.273⋅sin(q₁)⋅sin(q₂)⋅cos(q₃) + 0.311⋅sin(q₁)⋅sin(q₂) + 0.11⋅sin(q₁) + 0.273⋅ ↪
⎢                                                                              ↪
⎢              atan2(sin(q₁)⋅cos(q₃) + sin(q₂)⋅sin(q₃)⋅cos(q₁), sin(q₃)⋅cos(q₂ ↪
⎢                                                                              ↪
⎢                      -asin(sin(q₁)⋅sin(q₂)⋅sin(q₃) - cos(q₁)⋅cos(q₃))        ↪
⎢                                                                              ↪
⎣              atan2(sin(q₁)⋅cos(q₂), sin(q₁)⋅sin(q₂)⋅cos(q₃) + sin(q₃)⋅cos(q₁ ↪

↪                 ⎤
↪                 ⎥
↪ ) + 0.11

In [ ]:
# Forward kinematics function of the combined kinematics (in the inertial frame)
T_Ie = T_Ib * T_be
FK_centralised = sp.Matrix([T_Ie[0,3],
                T_Ie[1,3],
                T_Ie[2,3],
                sp.atan2(T_Ie[1,0], T_Ie[0,0]), # yaw
                -sp.asin(T_Ie[2,0]), # pitch
                sp.atan2(T_Ie[2,1], T_Ie[2,2])]) # roll
FK_centralised = sp.trigsimp(sp.expand(FK_centralised))

sp.pprint("State dependencies of centralised kinematics")
print(f"xe = f({FK_centralised[0,0].free_symbols})")
print(f"ye = f({FK_centralised[1,0].free_symbols})")
print(f"ze = f({FK_centralised[2,0].free_symbols})")
print(f"yawe = f({FK_centralised[3,0].free_symbols})")
print(f"pitche = f({FK_centralised[4,0].free_symbols})")
print(f"rolle = f({FK_centralised[5,0].free_symbols})")

sp.pprint("Denominators for yaw and roll")
sp.pprint(T_be[0,0])
sp.pprint(T_be[2,2])

# Jacobian calculation for velocity
J_A_centralised = FK_centralised.jacobian([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

print(f"Centralised Jacobian shape: {J_A_centralised.shape}")

# Instantaneous states
xb_inst = 0.0
yb_inst = 0.0
zb_inst = 0.0
yaw_inst = 0.0
pitch_inst = 0.0
roll_inst = 0.0
q1_inst = -sp.pi/2
q2_inst = 0
q3_inst = sp.pi/2

# Substitute the instantaneous states
J_A_centralised_inst = J_A_centralised.subs([(x_b, xb_inst), (y_b, yb_inst), (z_b, zb_inst), (yaw, yaw_inst), (pitch, pitch_inst), (roll, roll_inst), (q1, q1_inst), (q2, q2_inst), (q3, q3_inst)]).evalf()
sp.pprint(J_A_centralised_inst)
print(J_A_centralised_inst.rank())

# Evaluate the FK at this state
FK_centralised_inst = FK_centralised.subs([(x_b, xb_inst), (y_b, yb_inst), (z_b, zb_inst), (yaw, yaw_inst), (pitch, pitch_inst), (roll, roll_inst), (q1, q1_inst), (q2, q2_inst), (q3, q3_inst)]).evalf()
sp.pprint(FK_centralised_inst)

State dependencies of centralised kinematics
xe = f({q_3, q_2, pitch, q_1, yaw, x_b, roll})
ye = f({y_b, q_3, q_2, pitch, q_1, yaw, roll})
ze = f({q_3, q_2, pitch, q_1, z_b, roll})
yawe = f({q_3, q_2, pitch, q_1, yaw, roll})
pitche = f({q_3, q_2, pitch, roll, q_1})
rolle = f({q_3, q_2, pitch, q_1, roll})
Denominators for yaw and roll
sin(q₃)⋅cos(q₂)
sin(q₁)⋅sin(q₂)⋅cos(q₃) + sin(q₃)⋅cos(q₁)
Centralised Jacobian shape: (6, 9)
⎡1.0   0    0   -0.273  -0.421    0      0    -0.311    0  ⎤
⎢                                                          ⎥
⎢ 0   1.0   0     0       0     0.421  0.421    0       0  ⎥
⎢                                                          ⎥
⎢ 0    0   1.0    0       0     0.273  0.273    0     0.273⎥
⎢                                                          ⎥
⎢ 0    0    0    1.0     zoo      0      0     zoo      0  ⎥
⎢                                                          ⎥
⎢ 0    0    0     0       0      nan    nan     0      nan ⎥
⎢                     